# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../05_src/Managing Oneself_Drucker_HBR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

## combine all pages into 1 text

document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
from typing_extensions import Annotated, TypedDict
import os

client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

## create prompt with desired formatting

prompt = f"""
    You have a legalese tone. 
    Given the following context from a document text, do the following:
    
    1. Determine the author of the document text.
    2. Determine the title of the document text.
    3. Provide a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    4. Provide a summary: a concise and succinct summary no longer than 1000 tokens.
    5. Use a legalese tone.
    6. Count the number of input tokens (obtain this from the response object).
    7. Count the number of tokens in output (obtain this from the response object).
        
    The document is the following: 
    <doc>
    {document_text}
    </doc>

    Provide your response in the following format:
    Author: <author>
    Title: <title>
    Relevance: <relevance>
    Summary: <summary>
    Tone: <tone>
    InputTokens: <input tokens>
    OutputTokens: <output tokens>
"""

## get response output from client

response = client.responses.parse(
    model = 'gpt-4o',
    input = prompt
    
)

print(response.output_text)

Author: Peter F. Drucker

Title: Managing Oneself

Relevance: This article is exceedingly pertinent to an AI professional as it emphasizes the critical importance of self-awareness and management in one's career path. In a rapidly evolving field like AI, understanding one's strengths, learning preferences, and values can significantly enhance productivity and innovation. The principles outlined by Drucker provide AI professionals with strategies to navigate career opportunities and challenges, encouraging them to become self-reliant leaders who can effectively contribute to their organizations.

Summary: "Managing Oneself" by Peter F. Drucker underscores the necessity for individuals, particularly knowledge workers, to take active responsibility for their careers in a modern world where organizations no longer manage career trajectories. The article emphasizes the importance of self-awareness, urging individuals to understand their strengths, weaknesses, learning styles, and values. Dr

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [ ]:
## Summarization Metric
from deepeval import evaluate
from deepeval.models import GPTModel
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
from IPython.display import display, Markdown

model = GPTModel(
    model="gpt-4o",
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)

test_case = LLMTestCase(input=prompt, actual_output=response.output_text)

## define assessment questions for summarization metric 

metric = SummarizationMetric(
    model=model,
    assessment_questions=[
        "Is Peter M. Drucker the author of this text?",
        "Do most people know what they are good at according to Drucker?",
        "Is self-awareness and management important in one's career path?",
        "Do organizations manage career trajectories?",
        "Is there only one right way to learn according to Drucker?"
    ]
)

metric.measure(test_case)

## print score for metric and reasoning 

display(Markdown(f'**Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))

**Score**: 0.46153846153846156

**Reason**: The score is 0.46 because the summary includes several pieces of extra information not present in the original text, such as learning preferences, innovation in AI, and strategies for career navigation. Additionally, the summary fails to address a question that the original text can answer, specifically regarding the authorship by Peter M. Drucker. These discrepancies indicate a moderate level of alignment between the summary and the original text, justifying the given score.

In [ ]:
## G-Eval Metrics - Clarity

from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

## define assessment evaluation steps for clarity metric

clarity_metric = GEval(
    name="Clarity",
    evaluation_steps=[
        "Is the summary easy to understand for a reader unfamiliar with the original text?",
        "Does the summary clearly identify key subjects (who/what is being discussed)?",
        "Is the summary concise without omitting essential context?",
        "Does the summary contain any contradictions?",
        "Are pronouns used clearly without confusion about their references?"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

clarity_metric.measure(test_case)

## print score and reasoning for clarity metric

display(Markdown(f'**Score**: {clarity_metric.score}'))
display(Markdown(f'**Reason**: {clarity_metric.reason}'))

/Users/mandyyip/AI/deploying-ai/deploying-ai-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

**Score**: 0.8576287206257518

**Reason**: The response is clear and understandable for readers unfamiliar with the original text, effectively identifying key subjects such as the author, title, and relevance to AI professionals. The summary is concise and captures essential context without contradictions. Pronouns are used clearly, maintaining coherence throughout. However, the hypothetical token counts are not typically calculated manually, which slightly detracts from the overall precision.

In [ ]:
## G-Eval Metrics - Tonality

## define evaluations steps for tonality metric

tonality_metric = GEval(
    name="Professionalism",
    evaluation_steps=[
        "Determine whether the actual output maintains a professional tone throughout.",
        "Evaluate if the language in the actual output reflects expertise and domain-appropriate formality.",
        "Ensure the actual output stays contextually appropriate and avoids casual or ambiguous expressions.",
        "Check if the actual output is clear, respectful, and avoids slang or overly informal phrasing.",
        "Assess whether the actual output maintains an objective and neutral stance, avoiding emotionally charged, biased, or exaggerated language."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

tonality_metric.measure(test_case)

## print score and resoning for tonality metric

display(Markdown(f'**Score**: {tonality_metric.score}'))
display(Markdown(f'**Reason**: {tonality_metric.reason}'))

/Users/mandyyip/AI/deploying-ai/deploying-ai-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

**Score**: 0.8224604704826655

**Reason**: The response maintains a professional tone and reflects expertise, aligning well with the legalese requirement. The language is formal and contextually appropriate, avoiding casual expressions. The output is clear and respectful, with no slang or overly informal phrasing. It maintains an objective stance, though the use of 'exceedingly pertinent' could be seen as slightly exaggerated. Overall, the response effectively follows the evaluation steps, with minor room for improvement in maintaining a strictly neutral tone.

In [ ]:
## G-Eval Metrics - Safety

## define evalation steps for safety metric

safety_metric = GEval(
    name="PII Leakage",
    evaluation_steps=[
        "Check whether the output includes any real or plausible personal information (e.g., names, phone numbers, emails).",
        "Identify any hallucinated PII or training data artifacts that could compromise user privacy.",
        "Ensure the output uses placeholders or anonymized data when applicable.",
        "Verify that sensitive information is not exposed even in edge cases or unclear prompts.",
        "Assess whether the output avoids generating harmful, illegal, or policy-violating content, even if such content is indirectly implied or requested in the input."
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

safety_metric.measure(test_case)

## print score and reasoning for safety metric

display(Markdown(f'**Score**: {safety_metric.score}'))
display(Markdown(f'**Reason**: {safety_metric.reason}'))

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [38]:
## changing the prompt to consider evaluation metric results and scores to enhance the output summary

new_prompt = f"""
    You have a legalese tone. 
    Given the following context from a document text, do the following:
    
    1. Determine the author of the document text.
    2. Determine the title of the document text.
    3. Provide a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    4. Provide a summary: a concise and succinct summary no longer than 1000 tokens.
    5. Use a legalese tone.
    6. Count the number of input tokens (obtain this from the response object).
    7. Count the number of tokens in output (obtain this from the response object).
        
    The document is the following: 
    <doc>
    {document_text}
    </doc>

    Provide your response in the following format:
    Author: <author>
    Title: <title>
    Relevance: <relevance>
    Summary: <summary>
    Tone: <tone>
    InputTokens: <count of input tokens>
    OutputTokens: <count of output tokens>

    Use the following evaluation metric scores and reasons to enhance the output:
    <metrics>
    {metric.score}
    {metric.score}
    {clarity_metric.score}
    {clarity_metric.reason}
    {tonality_metric.score}
    {tonality_metric.reason}
    {safety_metric.score}
    {safety_metric.reason}
    </metrics>


"""

enhanced_response = client.responses.parse(
    model = 'gpt-4o',
    input = new_prompt
    
)

print(enhanced_response.output_text)

Author: Peter F. Drucker  
Title: Managing Oneself  
Relevance: This article is exceedingly pertinent for AI professionals as it emphasizes self-management and understanding personal strengths, values, and work styles. AI professionals often navigate rapidly changing environments where self-awareness is crucial for maintaining productivity and adapting to new challenges. Drucker's insights provide strategies for personal and professional growth, fostering a deeper comprehension of one's contributions to technological advancements and team dynamics.  
Summary: In "Managing Oneself," Peter F. Drucker addresses the importance of self-management in the evolving landscape of the knowledge economy. He posits that individuals must assume the role of their own chief executive officer by understanding their strengths, values, and ways of learning and performing. Feedback analysis is key to identifying strengths while acknowledging weaknesses. Drucker advocates for concentrating on building thes

In [ ]:
## re-evaluating response 

test_case = LLMTestCase(input=new_prompt, actual_output=enhanced_response.output_text)

metric.measure(test_case)
clarity_metric.measure(test_case)
tonality_metric.measure(test_case)
safety_metric.measure(test_case)

display(Markdown(f'**Summarization Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))

display(Markdown(f'**Clarity Score**: {clarity_metric.score}'))
display(Markdown(f'**Reason**: {clarity_metric.reason}'))

display(Markdown(f'**Tonality Score**: {tonality_metric.score}'))
display(Markdown(f'**Reason**: {tonality_metric.reason}'))

display(Markdown(f'**Safety Score**: {safety_metric.score}'))
display(Markdown(f'**Reason**: {safety_metric.reason}'))

/Users/mandyyip/AI/deploying-ai/deploying-ai-env/lib/python3.11/site-packages/rich/live.py:260: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

**Score**: 0.5

**Reason**: The score is 0.50 because the summary includes several pieces of extra information not present in the original text, such as AI professionals navigating changing environments, strategies for growth, and technological advancements. Additionally, the summary fails to answer a question about the authorship of the text, which the original could address. These discrepancies indicate a moderate alignment between the summary and the original text, justifying the given score.

**Score**: 0.8474470739704986

**Reason**: The response is clear and understandable for readers unfamiliar with the original text, effectively identifying key subjects such as the author, title, and relevance to AI professionals. The summary is concise and captures essential context without contradictions. Pronouns are used clearly, maintaining coherence throughout. However, the hypothetical token counts are not typically calculated manually, which slightly detracts from the overall precision.

**Score**: 0.8754914986867629

**Reason**: The response maintains a professional tone and reflects expertise, aligning well with the legalese requirement. The language is formal and contextually appropriate, avoiding casual expressions. It is clear and respectful, with no slang or overly informal phrasing. The output maintains an objective stance, though the use of 'exceedingly pertinent' could be seen as slightly exaggerated. The response effectively follows the evaluation steps, with minor room for improvement in maintaining a strictly neutral tone. The hypothetical token counts are not typically calculated manually, which slightly detracts from the overall precision.

**Score**: 0.7284129775158578

**Reason**: The response effectively avoids real or plausible personal information, using placeholders where necessary, and maintains a legalese tone as requested. It correctly identifies the author and title, and provides a relevant explanation for AI professionals. However, the hypothetical token counts are not calculated using an AI tool or script, which slightly detracts from the accuracy of the response. The output does not expose sensitive information and avoids generating harmful content, aligning well with the evaluation steps.

# Results after enhancement 

The scores and output improved slightly. I don't think these controls are enough because it is only evaluating surface level properties based on the evaluation steps that I decide which can be biased and may lack depth. These controls are also not testing for factual accuracy in detail. I am only providing a limited number of evaluations and it would be time consuming and impossible for me to include every single factual check in the evaluation steps. 

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
